In [9]:
import pandas as pd
from diffprivlib.tools import mean

import warnings # ignore errors.
warnings.filterwarnings("ignore")

data = pd.read_csv(r"Write the path of your data",delimiter=",") # Specify the path of file. 

data["TIME"] = pd.to_datetime(data["TIME"])

data_sorted = data.sort_values(by=["ID","TIME"])

df_new = pd.DataFrame()
for j in data.ID.unique():
    data_id = data_sorted.loc[data_sorted.ID==j]   
    data_id.reset_index(drop=True, inplace=True)
    data_id["new_X"] = None
    data_id["new_Y"] = None
    data_id["new_X_DP"] = None
    data_id["new_Y_DP"] = None    
    for i in range (len(data_id)-1):
        if(i<len(data_id)):
            data_id["new_X"][i]= (data_id["X"][i] + data_id["X"][i+1] ) / 2.0
            data_id["new_Y"][i]= (data_id["Y"][i] + data_id["Y"][i+1] ) / 2.0
            
            if(data_id["X"][i]<data_id["X"][i+1]): # Boundary Control. Choosing larger X value for Differential Privacy.
                b1x = data_id["X"][i]
                b2x = data_id["X"][i+1]
            else:
                b1x = data_id["X"][i+1]
                b2x = data_id["X"][i]
                
            if(data_id["Y"][i]<data_id["Y"][i+1]): # Boundary Control. Choosing larger Y value for Differential Privacy.
                b1y = data_id["Y"][i]
                b2y = data_id["Y"][i+1]
            else:
                b1y = data_id["Y"][i+1]
                b2y = data_id["Y"][i]
                
            data_id["new_X_DP"][i] =  mean(data_id["new_X"][i], epsilon=2.0,bounds=(b1x, b2x)) # you can try different epsilon values.
            data_id["new_Y_DP"][i] =  mean(data_id["new_Y"][i], epsilon=2.0,bounds=(b1y, b2y))
            
    df_new = df_new.append(data_id, ignore_index=False)


df_new.dropna(inplace=True)
df_new.reset_index(inplace=True,drop=True)


df_new.to_csv("Data with Anonymization.xls")